# Redes Neuronales con Keras y Scikit
## Proyecto precios vehiculos usados
**DataSet**: https://drive.google.com/drive/u/1/folders/1C9gMArnhfKOXoJFPkVT5rOK6sFnXj9Uo  
**Código**: https://drive.google.com/drive/u/1/folders/1JsEnifaNjxR1VmzIFo07u5M_9wEdEfnh

## Contenido
- Contexto Analítico y exploración de datos
- Ingeniería de datos
- Diseño, Entrenamiento y Evaluación de la RN

## Contexto analítico y exploración de datos
El mercado de autos usados es reconocido por ser un sector economico muy competido con un centenar de compañias que luchan por hacerse con una porción de la torta. El precio de los autos se devalua año año debido a multiples factores y determinar el precio correcto es clave para las compañias para lograr competir en el mercado. En este caso se requiere implementar una red neural que permita determinar el valor más justo para los vehiculos dependiento de sus atributos.  

Se cuenta con un dataset (Craiglist_Cars.csv) que serán cargados directamente a Colab.

In [ ]:
#!pip install google-colab

In [1]:
#Importamos las librerias necesarias
import pandas as pd
import numpy as np
import io
#from google.colab import files #Librería necesaria para interactuar con archivos en Colab

In [2]:
#uploaded = files.upload()
#cars = pd.read_csv(io.StringIO(uploaded['Craiglist_Cars.csv'].decode('utf-8')), sep = ',' )
cars = pd.read_csv('Craiglist_Cars.csv', sep = ',' )
cars.head(3)

,year,manufacturer,condition,cylinders,fuel,title_status,transmission,drive,size,type,paint_color,price
0,2009,chevrolet,good,8 cylinders,gas,clean,automatic,rwd,full-size,SUV,white,9000
1,2002,gmc,good,8 cylinders,gas,clean,automatic,4wd,NaN,pickup,white,6000
2,2007,pontiac,excellent,4 cylinders,gas,clean,automatic,fwd,compact,convertible,red,7000


## 1. Evalua la cantidad, tipo y completitud de las variables disponibles

In [3]:
print(cars.shape)
100*cars.isnull().sum()/cars.shape[0]

(434542, 12)


year             0.000000
manufacturer     3.646138
condition       36.860649
cylinders       28.306355
fuel             0.839505
title_status     0.662997
transmission     0.989087
drive           13.280189
size            58.133621
type            11.566661
paint_color     19.734571
price            0.000000
dtype: float64

In [4]:
types = pd.DataFrame(cars.dtypes)
print("Tipos de variables",types.groupby(0).size())

Tipos de variables 0
int64      2
object    10
dtype: int64


In [5]:
categoricas = types.index[types[0] == 'O'].values
for line in categoricas:
    print("La variable "+ line +" contiene:",str(len(cars[line].unique()))+" distinct values")

La variable manufacturer contiene: 43 distinct values
La variable condition contiene: 7 distinct values
La variable cylinders contiene: 9 distinct values
La variable fuel contiene: 6 distinct values
La variable title_status contiene: 7 distinct values
La variable transmission contiene: 4 distinct values
La variable drive contiene: 4 distinct values
La variable size contiene: 5 distinct values
La variable type contiene: 14 distinct values
La variable paint_color contiene: 13 distinct values


In [6]:
for i in categoricas:
    print("Distribución de ",i,": \n", 100*cars[i].value_counts()/cars.shape[0],"\n")

Distribución de  manufacturer : 
 ford               17.966273
chevrolet          14.873591
toyota              7.667383
nissan              5.094789
honda               5.086045
jeep                4.915060
ram                 4.827381
gmc                 4.297628
dodge               3.775699
bmw                 2.687887
hyundai             2.462133
subaru              2.127988
mercedes-benz       2.062401
volkswagen          2.023280
chrysler            1.783947
kia                 1.712378
cadillac            1.504343
buick               1.329906
lexus               1.304592
mazda               1.288023
audi                1.187917
acura               0.865509
infiniti            0.783814
pontiac             0.766784
lincoln             0.685319
volvo               0.590507
mitsubishi          0.550004
mini                0.441154
rover               0.433330
mercury             0.412848
saturn              0.377639
jaguar              0.226906
fiat                0.153265
harley-da

## Ingeniería de datos
Ahora se debe preparar la información para poder alimentar la red neuronal.

## 2. Implementa estrategías para tratar la información nula en las variables cuya tasa de nulos sea máximo el 10%

In [7]:
#Reemplazamos el tipo de actividad por la moda 
cars["fuel"] = cars["fuel"].fillna(cars["fuel"].mode()[0])
cars["title_status"] = cars["title_status"].fillna(cars["title_status"].mode()[0])
cars["transmission"] = cars["transmission"].fillna(cars["transmission"].mode()[0])
cars["manufacturer"] = cars["manufacturer"].fillna(cars["manufacturer"].mode()[0])
#Verificamos el cambio
100*cars.isnull().sum()/cars.shape[0]

year             0.000000
manufacturer     0.000000
condition       36.860649
cylinders       28.306355
fuel             0.000000
title_status     0.000000
transmission     0.000000
drive           13.280189
size            58.133621
type            11.566661
paint_color     19.734571
price            0.000000
dtype: float64

Luego del procedimiento anterior se debe proceder a convertir las variables categoricas en variables numericas. Durante el curso implementamos un método de One Hot Encoding disponible en Scikit Learn. En este caso utilizaremos una funcionalidad embedida en Pandas denominada "get_dummies":

In [8]:
df= cars.copy()
for col in categoricas:
    df = pd.concat([df, (pd.get_dummies(df[col])).astype(int)], axis=1)
    df.drop(columns=[col],inplace=True)
#Al crear las variables dummies se crean varias columnas referentes a categorias
# 'other' que no aportan ningún valor al dataset por lo cual las eliminamos
df.drop('other', axis=1, inplace=True)
print(df.shape)
df.head(3)

(434542, 100)


,year,price,acura,alfa-romeo,aston-martin,audi,bmw,buick,cadillac,chevrolet,...,brown,custom,green,grey,orange,purple,red,silver,white,yellow
0,2009,9000,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,1,0
1,2002,6000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2007,7000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [9]:
types = pd.DataFrame(df.dtypes)
print("Tipos de variables",types.groupby(0).size())

Tipos de variables 0
int32    98
int64     2
dtype: int64


In [17]:
numeric_columns = list(set(types.index[types[0] =="int64"].values) - set(["price"]))
variables_consolidadas = df[numeric_columns]
objetivo = df["price"] #Variable objetivo de nuestra regresion.

variables_consolidadas.head(3)

,year
0,2009
1,2002
2,2007


## 3. Separa el set de datos consolidados en 3 sets (entrenamiento, prueba y validación) de acuerdo con las recomendaciones vistas en el curso.

In [11]:
from sklearn.model_selection import train_test_split
#Entrenamiento y prueba con relación 80:20
x_train, x_test, y_train, y_test = train_test_split(variables_consolidadas,objetivo, test_size=0.2,random_state=2020)
#Entrenamiento y validación con relación 90:10
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train, test_size=0.1, random_state=2020)

y_train=y_train.values.reshape(-1,1)
y_test=y_test.values.reshape(-1,1)
y_val=y_val.values.reshape(-1,1)
#Se hace este reshape con la finalidad de tener un vector de una columna, inicialmente es un vector de una fila
print("Shape of x_train:",x_train.shape)
print("Shape of x_test:",x_test.shape)
print("Shape of x_val:",x_val.shape)
print("Shape of y_train:",y_train.shape)
print("Shape of y_test:",y_test.shape)
print("Shape of y_val:",y_val.shape)

Shape of x_train: (312869, 1)
Shape of x_test: (86909, 1)
Shape of x_val: (34764, 1)
Shape of y_train: (312869, 1)
Shape of y_test: (86909, 1)
Shape of y_val: (34764, 1)


## Diseño, Entrenamiento y Evaluación de la RN
Una vez consolidado los sets de información de entrenamiento, validacion y pruebas ya podemos iniciar a modelar nuestra red neuronal con las siguientes consideraciones:

- Realiza la prueba con un par de arquitecturas iniciales.
- Evalua el desempeño de la red.
- Si el desempeño es bajo vuelve a la información y prueba estrategias de estandarización de la información.

## 4. Implementa una red neuronal cuyas pérdidas (MSE) con el set de prueba sea menor a 0.40.

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(x_train)
x_train_scaled = scaler.transform(x_train)
x_val_scaled = scaler.transform(x_val)
x_test_scaled = scaler.transform(x_test)

scaler1 = StandardScaler()
scaler1.fit(y_train)
y_train_scaled = scaler1.transform(y_train)
y_val_scaled = scaler1.transform(y_val)
y_test_scaled = scaler1.transform(y_test)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.utils import plot_model

model = Sequential()
model.add(Dense(256,input_dim = x_train.shape[1],activation="relu"))
model.add(Dense(128,activation="relu"))
model.add(Dense(128,activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1,activation = "linear")) 
model.compile(optimizer = "adam",loss="mse",metrics=["mean_absolute_error"])

print(model.summary())

In [ ]:
model.fit(x_train_scaled,y_train_scaled, validation_data = (x_val_scaled,y_val_scaled),epochs=50, batch_size=1024)

Ahora realiza la evaluación del modelo con el set de entrenamiento.

In [ ]:
result = model.evaluate(x_test_scaled,y_test_scaled)
for i in range(len(model.metrics_names)):
    print("Metric ",model.metrics_names[i],":",
str(round(result[i],2)))

Si tu modelo cumple con el requerimiento guardalo con el siguiente comando:

In [ ]:
model.save('PrediccionPrecios.h5')

## 5. Realiza un gráfico que evidencia la evolución de la función de pérdidas a traves de las distintas épocas de entrenamiento.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.figure(figsize=(13,6))
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.title("Pérdidas del modelo con set de entrenamiento y pruebas por época")
plt.ylabel('MSE')
plt.xlabel('Épocas')
plt.legend(['Entrenamiento', 'Validación'], loc='upper right')
plt.show()

Trata de realizar predicciones con el modelo

In [ ]:
real=pd.DataFrame(y_train)
predic=model.predict(pd.DataFrame(x_train_scaled))
valores_reescalados = scaler1.inverse_transform(predic)
pred_escal =pd.DataFrame(valores_reescalados)
# Muestra los valores reales y las predicciones
for i in range(0,5):
	print("Real=%s, Prediccion=%s" % (real[0][i], pred_escal[0][i]))